In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../")

In [3]:
from ease_recommender import *
from npmi_recommender import *

import pickle as p

In [4]:
print("loading cache data...")
D = p.load(open("cached_data/movie_lens_preprocessed.p", "rb"))

row = D["userId"]
col = D["movieId"]
data = D["rating"]

movies = D["movies"]

print("done")

loading cache data...
done


In [5]:
def find_match_using_terms(terms, movies=movies, case_insensitive=False):
    assert type(terms) in (list, tuple, set)
    
    title = movies.title
    if case_insensitive:
        title = title.str.lower()
    
    matches = True
    for term in terms:
        matches &= title.str.contains(term)
        
    matches = np.where(matches)[0]

    if len(matches) > 1:
        raise Exception("Multiple matches found, filter down to a single match", matches)
        
    return matches[0]

In [6]:
# hyper parameter like/love threshold
rating_threshold = 0

mask = data >= rating_threshold

mat = csr_matrix((mask[mask], (row[mask], col[mask]))).astype(np.int64)

In [7]:
a = find_match_using_terms(["Sense and Sensibility", "1995"])

movies.loc[a]

title         Sense and Sensibility (1995)
genres                       Drama|Romance
imdbId                              114388
tmdbId                              4584.0
avg_rating                        7.891793
num_votes                             8811
Name: 16, dtype: object

In [8]:
b = find_match_using_terms(["Pride and Prejudice", "1995"])

movies.loc[b]

title         Pride and Prejudice (1995)
genres                     Drama|Romance
imdbId                            112130
tmdbId                          164721.0
avg_rating                       7.98947
num_votes                           1517
Name: 7228, dtype: object

In [9]:
lambda_ = optimize_lambda_using_a_to_b_matching(mat, a, b, fast_approximation=False)

lambda_: 1000000
error: 115.000001
lambda_: 100000.0
error: 44.00001
lambda_: 10000.0
error: 14.0001
lambda_: 1000.0
error: 13.001
lambda_: 100.0
error: 25.01
lambda_: 100.0
error: 25.01
lambda_: 1000.0
error: 13.001
lambda_: 10000.0
error: 14.0001
lambda_: 4437.694
error: 9.000225342261093
lambda_: 6562.305773596001
error: 11.000152385462442
lambda_: 4256.388734945126
error: 8.000234940946957
lambda_: 3012.5589554130756
error: 9.000331943711243
lambda_: 3725.134911961836
error: 8.000268446653244
lambda_: 3990.7658974177466
error: 8.000250578466817
lambda_: 4154.929842186143
error: 8.000240677950767
lambda_: 4325.641181817076
error: 8.000231179600426
lambda_: 4368.441548567135
error: 8.00022891458862
lambda_: 4394.893630431141
error: 8.000227536792488
lambda_: 4411.24191639388
error: 8.000226693529612
lambda_: 4421.345712960576
error: 8.000226175482517
lambda_: 4427.590202767877
error: 8.000225856493985
lambda_: 4431.449509781442
error: 8.000225659797724
lambda_: 4433.834692732265
erro

In [10]:
temp = 1

In [11]:
# check error for NPMI in comparison
a_to_b_error_metric_npmi(mat, a, b, temp)

temp: 1
error: 479.0


479.0

In [12]:
top_k = 20

In [13]:
# using EASE

similarity_scores = calculate_ease_for_item_cg(mat, a, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
815,Emma (1996),Comedy|Drama|Romance,116191,3573.0,7.485110,2392
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
492,Much Ado About Nothing (1993),Comedy|Romance,107616,11971.0,7.734885,4635
510,"Remains of the Day, The (1993)",Drama|Romance,107943,1245.0,7.783682,3555
258,Little Women (1994),Drama,110367,9587.0,7.199093,2599
57,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance,110877,11010.0,7.927828,5037
35,Dead Man Walking (1995),Crime|Drama,112818,687.0,7.847888,7682
7228,Pride and Prejudice (1995),Drama|Romance,112130,164721.0,7.989470,1517
352,Four Weddings and a Funeral (1994),Comedy|Romance,109831,712.0,7.284194,6370


In [14]:
# using EASE

similarity_scores = calculate_ease_for_item_cg(mat, b, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
9966,Pride & Prejudice (2005),Drama|Romance,414387,4348.0,7.703508,3062
15953,North & South (2004),Drama|Romance,417349,147269.0,8.057732,243
16,Sense and Sensibility (1995),Drama|Romance,114388,4584.0,7.891793,8811
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
13373,Persuasion (2007),Drama|Romance,844330,13949.0,7.719715,150
14258,"King's Speech, The (2010)",Drama,1504320,45269.0,7.814929,7203
12497,"Young Victoria, The (2009)",Drama|Romance,962736,18320.0,7.472767,247
2960,Mansfield Park (1999),Comedy|Drama|Romance,178737,10399.0,7.523058,404
14718,Jane Eyre (2011),Drama|Romance,1229822,38684.0,7.451439,288


In [15]:
# using normalized pointwise mutual information

similarity_scores = npmi_batch(mat, a, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
815,Emma (1996),Comedy|Drama|Romance,116191,3573.0,7.485110,2392
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
492,Much Ado About Nothing (1993),Comedy|Romance,107616,11971.0,7.734885,4635
510,"Remains of the Day, The (1993)",Drama|Romance,107943,1245.0,7.783682,3555
352,Four Weddings and a Funeral (1994),Comedy|Romance,109831,712.0,7.284194,6370
262,Like Water for Chocolate (Como agua para choco...,Drama|Fantasy|Romance,103994,18183.0,7.827167,4205
258,Little Women (1994),Drama,110367,9587.0,7.199093,2599
35,Dead Man Walking (1995),Crime|Drama,112818,687.0,7.847888,7682
57,"Postman, The (Postino, Il) (1994)",Comedy|Drama|Romance,110877,11010.0,7.927828,5037


In [16]:
# using normalized pointwise mutual information

similarity_scores = npmi_batch(mat, b, lambda_)

top_k_matches = movies.loc[np.argsort(-similarity_scores)[:top_k].tolist()]

top_k_matches

,title,genres,imdbId,tmdbId,avg_rating,num_votes
movieId,,,,,,
15953,North & South (2004),Drama|Romance,417349,147269.0,8.057732,243
13373,Persuasion (2007),Drama|Romance,844330,13949.0,7.719715,150
9966,Pride & Prejudice (2005),Drama|Romance,414387,4348.0,7.703508,3062
15308,Northanger Abbey (2007),Drama|Romance,844794,18093.0,7.429658,84
12497,"Young Victoria, The (2009)",Drama|Romance,962736,18320.0,7.472767,247
16,Sense and Sensibility (1995),Drama|Romance,114388,4584.0,7.891793,8811
14718,Jane Eyre (2011),Drama|Romance,1229822,38684.0,7.451439,288
27,Persuasion (1995),Drama|Romance,114117,17015.0,8.076637,1327
10919,Becoming Jane (2007),Drama|Romance,416508,2977.0,7.129006,273
